# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data = pd.read_csv("output_data/cities.csv", index_col=0)
city_data.head()

,city,lat,lon,maxtemp,humidity,cloudiness,windspeed,country,datetime
0,jalu,29.03,21.55,59.20,35,0,11.54,LY,1607229534
1,butaritari,3.07,172.79,82.04,78,90,12.48,KI,1607229534
2,port elizabeth,-33.92,25.57,66.20,93,90,10.29,ZA,1607229449
3,yar-sale,66.83,70.83,11.26,94,92,10.40,RU,1607229534
4,lebu,-37.62,-73.65,52.09,92,96,12.15,CL,1607229534


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Google developer API key
from api_keys import g_key

# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = city_data[["lat", "lon"]].astype(float)
humidity = city_data["humidity"]

In [5]:
# Create the heat map
# Set the layout to see the majority of the data points without duplication
figure_layout = {
    'width': '1000px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px'
}
fig = gmaps.figure(center=(0.00, 0.00), zoom_level = 2, layout=figure_layout)

# add the heat layer showing cities with their humidity as weight.
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 5)
fig.add_layer(heat_layer)

# show the map
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='1000px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

My ideal weather conditions are:
* Between 40 and 50 degrees
* Medium humidity - no more than 50%
* Light breeze only (5 mph or less)


In [6]:
ideal_city_data = city_data.loc[((city_data["maxtemp"] >= 40) 
                                & (city_data["maxtemp"] <=50) 
                                & (city_data["humidity"] <= 50)
                                & (city_data["windspeed"] <= 5))]
ideal_city_data

,city,lat,lon,maxtemp,humidity,cloudiness,windspeed,country,datetime
133,saint george,37.10,-113.58,41.0,18,1,1.86,US,1607229515
254,henderson,36.04,-114.98,48.2,20,1,1.48,US,1607229350
433,seoul,37.57,126.98,50.0,45,40,3.36,KR,1607229380
487,ankang,32.68,109.02,43.0,33,97,0.51,CN,1607229621


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = ideal_city_data
hotel_df["Hotel Name"] = ""

target_radius = 5000
target_type = "lodging"

for index, row in hotel_df.iterrows():
    target_coordinates = str(row['lat']) + "," + str(row['lon'])
    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params).json()
    hotel_df.loc[index,["Hotel Name"]] = response["results"][0]["name"]
    hotel.append(response["results"][0]["name"])



<ipython-input-7-adb50f4039dc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""
C:\Users\julie\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,city,lat,lon,maxtemp,humidity,cloudiness,windspeed,country,datetime,Hotel Name
133,saint george,37.10,-113.58,41.0,18,1,1.86,US,1607229515,Best Western Plus Abbey Inn
254,henderson,36.04,-114.98,48.2,20,1,1.48,US,1607229350,Hawthorn Suites By Wyndham Las Vegas/Henderson
433,seoul,37.57,126.98,50.0,45,40,3.36,KR,1607229380,LOTTE HOTEL SEOUL
487,ankang,32.68,109.02,43.0,33,97,0.51,CN,1607229621,安琪酒店


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lon"]]

In [12]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, hotel_info)
fig.add_layer(markers)

# Display figure
fig


['\n<dl>\n<dt>Name</dt><dd>Best Western Plus Abbey Inn</dd>\n<dt>City</dt><dd>saint george</dd>\n<dt>Country</dt><dd>US</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Hawthorn Suites By Wyndham Las Vegas/Henderson</dd>\n<dt>City</dt><dd>henderson</dd>\n<dt>Country</dt><dd>US</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>LOTTE HOTEL SEOUL</dd>\n<dt>City</dt><dd>seoul</dd>\n<dt>Country</dt><dd>KR</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>安琪酒店</dd>\n<dt>City</dt><dd>ankang</dd>\n<dt>Country</dt><dd>CN</dd>\n</dl>\n']